In [1]:
import spectral.io.envi as envi

'''
Need to convert tif to img:

gdal_translate -of ENVI mod09q1_2015.tif mod09q1_2015.img

'''

'\nNeed to convert tif to img:\n\ngdal_translate -of ENVI mod09q1_2015.tif mod09q1_2015.img\n\n'

In [2]:
data1 = 'mod09q1_2015'

img = envi.open(data1 +'.hdr', data1 + '.img')
img = img[:,:,:]

img.shape

# rows, cols, bands

(2257, 3398, 40)

In [3]:
data1 = 'mtbs_AK_2015_clip'

mask = envi.open(data1 +'.hdr', data1 + '.img')
mask = mask[:,:,:]

mask.shape

(2257, 3398, 1)

In [4]:
import numpy as np

# number of pixels per category (0 = no fire, 1 = fire, 2 = background)
(unique, counts) = np.unique(mask, return_counts=True)
np.asarray((unique, counts)).T

array([[      0, 7411228],
       [      1,   29353],
       [      2,   47698],
       [      3,   72808],
       [      4,   82127],
       [      5,    1133],
       [      6,   24939]], dtype=int64)

In [5]:
import pandas as pd
l = mask.flatten()
df = pd.DataFrame(data=l, columns=['label'])

In [ ]:
df.head(2)

In [6]:
# loop through bands and add to df
for i in range(0, img.shape[2]):
    #print (i)
    b = img[:,:,i].flatten()
    df['b'+str(i)] = b

In [47]:
df2 = df.copy()
df2.head(2)
label = df2[['label']]
del df2['label']

In [48]:
# replace label
label

label = label.astype(int).replace(0,-1)
label = label.astype(int).replace(1,1)
label = label.astype(int).replace(2,1)
label = label.astype(int).replace(3,1)
label = label.astype(int).replace(4,1)
label = label.astype(int).replace(5,1)
label = label.astype(int).replace(6,1)

print(label.value_counts())

label
-1       7411228
 1        258058
dtype: int64


In [50]:
# set 0 to NaN
df2 = df2.replace(0,np.NaN)
df2['label'] = label
# drops NaNs
df2 = df2.dropna()
df2

,b0,b1,b2,b3,b4,b5,b6,b7,b8,b9,...,b31,b32,b33,b34,b35,b36,b37,b38,b39,label
3045,1776.0,2577.0,456.0,1477.0,446.0,1584.0,429.0,2258.0,551.0,2088.0,...,1674.0,490.0,1693.0,1832.0,2448.0,645.0,2072.0,4335.0,4793.0,-1
3046,1706.0,2430.0,542.0,1638.0,538.0,1656.0,504.0,2279.0,638.0,2286.0,...,1613.0,518.0,1698.0,1153.0,1807.0,1172.0,2510.0,4733.0,5163.0,-1
3047,1706.0,2430.0,542.0,1638.0,538.0,1656.0,504.0,2279.0,638.0,2286.0,...,1613.0,518.0,1698.0,1153.0,1807.0,1172.0,2510.0,4733.0,5163.0,-1
3048,1706.0,2430.0,542.0,1638.0,538.0,1656.0,504.0,2279.0,638.0,2286.0,...,1613.0,518.0,1698.0,1153.0,1807.0,1172.0,2510.0,4733.0,5163.0,-1
6437,1487.0,2218.0,460.0,1402.0,457.0,1470.0,430.0,2164.0,516.0,2144.0,...,1467.0,442.0,1532.0,3077.0,3683.0,1033.0,2090.0,2345.0,3458.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663782,2397.0,2872.0,1107.0,1889.0,2337.0,2754.0,594.0,1990.0,1780.0,7900.0,...,6075.0,1012.0,1867.0,2382.0,2446.0,7775.0,7843.0,1435.0,1666.0,-1
7663783,4055.0,3959.0,2539.0,3081.0,1742.0,2238.0,716.0,1926.0,1626.0,6448.0,...,5587.0,760.0,1436.0,3130.0,2956.0,5455.0,5299.0,1607.0,1910.0,-1
7666952,951.0,2394.0,789.0,2652.0,516.0,2300.0,574.0,2181.0,584.0,2798.0,...,2323.0,540.0,2059.0,2253.0,2951.0,634.0,2262.0,413.0,1733.0,-1
7666953,893.0,2367.0,795.0,2670.0,516.0,2276.0,587.0,2198.0,599.0,2784.0,...,2347.0,564.0,2178.0,2081.0,2817.0,657.0,2293.0,423.0,1749.0,-1


In [51]:
# switch column order
cols = df2.columns.tolist()
cols = cols[-1:] + cols[:-1]
df2 = df2[cols]
df2

,label,b0,b1,b2,b3,b4,b5,b6,b7,b8,...,b30,b31,b32,b33,b34,b35,b36,b37,b38,b39
3045,-1,1776.0,2577.0,456.0,1477.0,446.0,1584.0,429.0,2258.0,551.0,...,496.0,1674.0,490.0,1693.0,1832.0,2448.0,645.0,2072.0,4335.0,4793.0
3046,-1,1706.0,2430.0,542.0,1638.0,538.0,1656.0,504.0,2279.0,638.0,...,512.0,1613.0,518.0,1698.0,1153.0,1807.0,1172.0,2510.0,4733.0,5163.0
3047,-1,1706.0,2430.0,542.0,1638.0,538.0,1656.0,504.0,2279.0,638.0,...,512.0,1613.0,518.0,1698.0,1153.0,1807.0,1172.0,2510.0,4733.0,5163.0
3048,-1,1706.0,2430.0,542.0,1638.0,538.0,1656.0,504.0,2279.0,638.0,...,512.0,1613.0,518.0,1698.0,1153.0,1807.0,1172.0,2510.0,4733.0,5163.0
6437,-1,1487.0,2218.0,460.0,1402.0,457.0,1470.0,430.0,2164.0,516.0,...,439.0,1467.0,442.0,1532.0,3077.0,3683.0,1033.0,2090.0,2345.0,3458.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663782,-1,2397.0,2872.0,1107.0,1889.0,2337.0,2754.0,594.0,1990.0,1780.0,...,6252.0,6075.0,1012.0,1867.0,2382.0,2446.0,7775.0,7843.0,1435.0,1666.0
7663783,-1,4055.0,3959.0,2539.0,3081.0,1742.0,2238.0,716.0,1926.0,1626.0,...,5341.0,5587.0,760.0,1436.0,3130.0,2956.0,5455.0,5299.0,1607.0,1910.0
7666952,-1,951.0,2394.0,789.0,2652.0,516.0,2300.0,574.0,2181.0,584.0,...,483.0,2323.0,540.0,2059.0,2253.0,2951.0,634.0,2262.0,413.0,1733.0
7666953,-1,893.0,2367.0,795.0,2670.0,516.0,2276.0,587.0,2198.0,599.0,...,504.0,2347.0,564.0,2178.0,2081.0,2817.0,657.0,2293.0,423.0,1749.0


In [44]:
'''
-1: No Wildfire
 1: Wildfire
'''
print(df2.label.value_counts())
df2.to_csv('mod09q1_2015.csv',header=None, sep=',', index=None)

-1    6341040
 1     252414
Name: label, dtype: int64


# Get data ready for PERMON

In [52]:
import sys
import numpy as np

sys.path.insert(0, 'C:/Users/lql/Desktop/rs_petsc/repos/petsc/lib/petsc/bin')
import PetscBinaryIO

In [101]:
def load_data(p, x):
    xf=p+x
    io = PetscBinaryIO.PetscBinaryIO()
    sparse_data= io.readBinaryFile(xf, mattype='scipy.sparse')
    xdata = sparse_data[0].toarray()
    return xdata

# Open example file (for comparison)

In [109]:
path = 'C:/Users/lql/Desktop/rs_petsc/data/'
file = 'heart_scale.bin'

x = load_data(path, file)

In [112]:
sparse_data = io.readBinaryFile(path + file, mattype='scipy.sparse')
sparse_data

(<180x13 sparse matrix of type '<class 'numpy.float64'>'
 	with 2248 stored elements in Compressed Sparse Row format>,
 Vec([-1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,  1.,
       1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1., -1.,
       1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,
       1., -1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,
      -1.,  1., -1.,  1.,  1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1.,
       1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1., -1.,
      -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,  1., -1.,  1.,  1.,
       1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,
       1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1.,
       1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,
      -1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1., -1.,
       1.,  1., -1.,  1., -1.,  1., -1., -1.,

In [129]:
x2 = sparse_data[0]
type(x2)

scipy.sparse.csr.csr_matrix

In [128]:
y2 = sparse_data[1]
type(y2)

PetscBinaryIO.Vec

In [132]:
y2

Vec([-1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,  1.,
      1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1., -1.,
      1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,
      1., -1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,
     -1.,  1., -1.,  1.,  1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1.,
      1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1., -1.,
     -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,  1., -1.,  1.,  1.,
      1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,
      1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1.,
      1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,
     -1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1., -1.,
      1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,
      1., -1., -1., -1., -1.,  1.,  1.,  1., -1., -1., -1., -1.])

# Duplicate with RS data

In [113]:
 df2['label'].value_counts()

,label,b0,b1,b2,b3,b4,b5,b6,b7,b8,...,b30,b31,b32,b33,b34,b35,b36,b37,b38,b39
3045,-1,1776.0,2577.0,456.0,1477.0,446.0,1584.0,429.0,2258.0,551.0,...,496.0,1674.0,490.0,1693.0,1832.0,2448.0,645.0,2072.0,4335.0,4793.0
3046,-1,1706.0,2430.0,542.0,1638.0,538.0,1656.0,504.0,2279.0,638.0,...,512.0,1613.0,518.0,1698.0,1153.0,1807.0,1172.0,2510.0,4733.0,5163.0
3047,-1,1706.0,2430.0,542.0,1638.0,538.0,1656.0,504.0,2279.0,638.0,...,512.0,1613.0,518.0,1698.0,1153.0,1807.0,1172.0,2510.0,4733.0,5163.0
3048,-1,1706.0,2430.0,542.0,1638.0,538.0,1656.0,504.0,2279.0,638.0,...,512.0,1613.0,518.0,1698.0,1153.0,1807.0,1172.0,2510.0,4733.0,5163.0
6437,-1,1487.0,2218.0,460.0,1402.0,457.0,1470.0,430.0,2164.0,516.0,...,439.0,1467.0,442.0,1532.0,3077.0,3683.0,1033.0,2090.0,2345.0,3458.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663782,-1,2397.0,2872.0,1107.0,1889.0,2337.0,2754.0,594.0,1990.0,1780.0,...,6252.0,6075.0,1012.0,1867.0,2382.0,2446.0,7775.0,7843.0,1435.0,1666.0
7663783,-1,4055.0,3959.0,2539.0,3081.0,1742.0,2238.0,716.0,1926.0,1626.0,...,5341.0,5587.0,760.0,1436.0,3130.0,2956.0,5455.0,5299.0,1607.0,1910.0
7666952,-1,951.0,2394.0,789.0,2652.0,516.0,2300.0,574.0,2181.0,584.0,...,483.0,2323.0,540.0,2059.0,2253.0,2951.0,634.0,2262.0,413.0,1733.0
7666953,-1,893.0,2367.0,795.0,2670.0,516.0,2276.0,587.0,2198.0,599.0,...,504.0,2347.0,564.0,2178.0,2081.0,2817.0,657.0,2293.0,423.0,1749.0


In [122]:
df3 = df2.copy()
y = df3['label'].values
x = df3.values
x

array([[1776., 2577.,  456., ..., 2072., 4335., 4793.],
       [1706., 2430.,  542., ..., 2510., 4733., 5163.],
       [1706., 2430.,  542., ..., 2510., 4733., 5163.],
       ...,
       [ 951., 2394.,  789., ..., 2262.,  413., 1733.],
       [ 893., 2367.,  795., ..., 2293.,  423., 1749.],
       [ 893., 2367.,  795., ..., 2293.,  423., 1749.]])

In [119]:
y

array([-1, -1, -1, ..., -1, -1, -1])

In [123]:
from scipy import sparse

sX = sparse.csr_matrix(x)

In [124]:
sX

<6593454x40 sparse matrix of type '<class 'numpy.float64'>'
	with 263738160 stored elements in Compressed Sparse Row format>

In [134]:
sY = y.view(PetscBinaryIO.Vec)
type(sY)

PetscBinaryIO.Vec

In [136]:
permon_data = (sX, sY)
permon_data

(<6593454x40 sparse matrix of type '<class 'numpy.float64'>'
 	with 263738160 stored elements in Compressed Sparse Row format>,
 Vec([-1, -1, -1, ..., -1, -1, -1]))

In [137]:
sparse_data

(<180x13 sparse matrix of type '<class 'numpy.float64'>'
 	with 2248 stored elements in Compressed Sparse Row format>,
 Vec([-1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,  1.,
       1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1., -1.,
       1., -1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,
       1., -1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,
      -1.,  1., -1.,  1.,  1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1.,
       1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1., -1.,
      -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,  1., -1.,  1.,  1.,
       1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,
       1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1.,
       1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,
      -1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1., -1.,
       1.,  1., -1.,  1., -1.,  1., -1., -1.,

In [138]:
# export as PETSc binary
# permon_data
#io.readBinaryFile(path + file, mattype='scipy.sparse')
io.writeBinaryFile('file.bin', permon_data)